In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import pyarrow.parquet as pq

## Reading data

In [2]:
dir_name = 'C:\Cloud\OneDrive - Emory University\Papers\PASC Cardiometabolic Risk Factors'
outcome_df = pd.read_parquet(dir_name + '\working\models\pcra201_ipw for cohort membership data.parquet' )
outcome_df.head()

,ID,female,nhwhite,nhblack,hispanic,nhother,age,matchid,index_date,site,...,LOINC_48159_8_gtQ3,LOINC_27297_1_gtQ3,LOINC_50553_7_gtQ3,LOINC_53290_3_gtQ3,LOINC_49524_2_gtQ3,LOINC_29908_1_gtQ3,LOINC_49521_8_gtQ3,LOINC_62423_9_gtQ3,LOINC_24027_5_gtQ3,LOINC_11557_6_gtQ3
0,12MAR202320220187400000001,1.0,0.0,0.0,0.0,1.0,57.0,,2021-02-16,Source1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12MAR202320220187400000002,1.0,1.0,0.0,0.0,0.0,33.0,,2021-01-27,Source1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12MAR202320220187400000003,1.0,1.0,0.0,0.0,0.0,72.0,,2018-08-20,Source5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12MAR202320220187400000004,0.0,1.0,0.0,0.0,0.0,84.0,,2020-05-21,Source7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12MAR202320220187400000005,1.0,1.0,0.0,0.0,0.0,27.0,,2021-03-01,Source1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# The below lines were commented out after testing a simple model
# outcome_df = outcome_df.sample(frac=0.1, random_state=1)
# sample_outcome_df.shape
outcome_df.shape

(391009, 2980)

In [4]:
# https://stackoverflow.com/questions/11587782/creating-dummy-variables-in-pandas-for-python
outcome_df = pd.get_dummies(outcome_df,prefix='',prefix_sep='_',
                                           columns=['site','calendar_month','payer_type_primary','payer_type_secondary'],drop_first=True)
outcome_df.shape
# https://www.kdnuggets.com/2020/07/easy-guide-data-preprocessing-python.html
# Also lists one-hot encoding as an option

(391009, 3005)

## Code for Random Forest Classifier
https://scikit-learn.org/stable/modules/grid_search.html

In [5]:
rf = RandomForestClassifier(random_state=1)
# n_estimators = number of trees in the forest
# random_state = controls both the randomness of the bootstrapping of the samples used when building trees (if bootstrap=True), 
# and the sampling of the features to consider when looking for the best split at each node (if max_features < n_features)
min_samples_leaf = [10, 25, 50]
n_estimators = [2000]
# n_estimators = [5, 10] # Commented out after testing a simple model

random_grid = {'min_samples_leaf': min_samples_leaf,
               'n_estimators': n_estimators}
print(random_grid)


{'min_samples_leaf': [50, 100], 'n_estimators': [1000, 2000]}


## Train and test split


In [6]:
y = outcome_df['COHORT']
X = outcome_df.drop(['COHORT','ID','matchid','index_date'], axis=1)

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

## Grid Search
https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


In [8]:
gs_rf = GridSearchCV(rf,
                      param_grid=random_grid,
                      # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring
                      # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
                      scoring = make_scorer(recall_score, average='weighted'),
                      cv=5)
gs_rf.fit(X_train, y_train)
gs_rf.best_params_

{'min_samples_leaf': 50, 'n_estimators': 2000}

In [9]:
gs_rf.cv_results_

{'mean_fit_time': array([1942.14743724, 3241.71643682, 1388.7774426 , 3413.82132888]),
 'std_fit_time': array([352.57403128,  25.68461837,   3.53934382, 511.41694299]),
 'mean_score_time': array([20.43543072, 30.79085488, 12.96508884, 33.34874105]),
 'std_score_time': array([ 9.72365495,  0.95545831,  0.14377666, 10.36688444]),
 'param_min_samples_leaf': masked_array(data=[50, 50, 100, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 2000, 1000, 2000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 50, 'n_estimators': 1000},
  {'min_samples_leaf': 50, 'n_estimators': 2000},
  {'min_samples_leaf': 100, 'n_estimators': 1000},
  {'min_samples_leaf': 100, 'n_estimators': 2000}],
 'split0_test_score': array([0.69938621, 0.69948211, 0.68476072, 0.68479269]),
 'split1_test_score': array([0.70008951, 0.7

In [10]:
gs_rf.best_score_

0.6994088970085883

In [11]:
gs_rf.score(X_test, y_test)

0.704764584025984

## Final Fit


In [12]:
y_pred = gs_rf.predict(X)
y_pred_proba = gs_rf.predict_proba(X)
y_pred_proba
# https://datatofish.com/numpy-array-to-pandas-dataframe/
pd.DataFrame(y_pred_proba,columns=['exposed','historical','unexposed']).to_csv(dir_name + '\working\models\pcra201_predicted probability for COHORT_min10_ntree2000.csv')

pd.crosstab(y,y_pred)


col_0,exposed,historical,unexposed
COHORT,,,
exposed,4894,3573,66750
historical,90,35204,39923
unexposed,495,3496,236584
